## Preprocess and motion correct 3D movies
Step 1 of the Caiman processing pipeline for multi-layer two-photon calcium imaging movies. Assume movie format acquired using the Scope setup (i.e. different layers arranged as mosaic on top of each other).

### Imports & Setup
The first cells import the various Python modules required by the notebook. In particular, a number of modules are imported from the Caiman package. In addition, we also setup the environment so that everything works as expected.

In [1]:
# Generic imports
import os, sys, platform, re, math, getpass, shutil, tempfile
import json, yaml
import time
import ipyparallel
import xml.etree.ElementTree as ET
import numpy as np
# import scipy
# import matplotlib.pyplot as plt
# import skimage.transform
# from tifffile import TiffFile, imread, imsave
from tifffile import imsave
from IPython.display import clear_output

# Import Bokeh library
# from bokeh.plotting import Figure, show
# from bokeh.layouts import gridplot
# from bokeh.models import Range1d, CrosshairTool, HoverTool, Legend
# from bokeh.io import output_notebook, export_svgs
# from bokeh.models.sources import ColumnDataSource
# from bokeh import palettes

# %matplotlib inline

In [2]:
# on Linux we have to add the caiman folder to Pythonpath
if platform.system() == 'Linux':
    sys.path.append(os.path.expanduser('~/caiman'))
# environment variables for parallel processing
os.environ['MKL_NUM_THREADS']='1'
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['VECLIB_MAXIMUM_THREADS']='1'

In [3]:
# Import CaImAn and custom functions
import caiman as cm
from caiman.motion_correction import MotionCorrect
from caiman.source_extraction.cnmf import params as params
import utils, mc_utils

In [4]:
# This has to be in a separate cell, otherwise it wont work.
# from bokeh import resources
# output_notebook(resources=resources.INLINE)

### Read parameters from config file

In [5]:
config_file = 'config.yml'
with open(config_file) as f:
    config = yaml.load(f, Loader=yaml.FullLoader)
print(config)

{'general': {'ncpus': 4}, 'data': {'connect_storage': 0, 'storage_adress': '//130.60.51.15/Neurophysiology-Storage2', 'storage_user': 'luetcke', 'mountpoint': '/home/luetcke/neurophys-storage', 'copy_to_temp': 0, 'data_folder': '/Users/Henry/Data/temp/Dendrites_Gwen', 'animal_folder': 'M5.2', 'day_folder': 20181211, 'area_folder': 'S1', 'group_id': 'G0', 'mc_output': 'rig'}, 'analysis': {'max_trials': 10, 'n_planes': 4, 'x_crop': 0.5, 'max_group_size': 999, 'remove_bad_frames': 1}, 'mc': {'niter_rig': 5, 'pw_rigid': False, 'splits_rig': 50, 'num_splits_to_process_rig': 'None', 'strides': [24, 24], 'overlaps': [24, 24], 'splits_els': 50, 'num_splits_to_process_els': [28, 'None'], 'upsample_factor_grid': 4, 'max_deviation_rigid': 10, 'border_nan': False}, 'metrics': {'winsize': 100, 'swap_dim': False, 'resize_fact_flow': 0.2, 'iters_flow': 3}}


### Setup cluster for parallel processing
This section starts the IPython cluster (ipcluster).

In [6]:
ncpus = config['general']['ncpus']

In [7]:
%%bash -s "$ncpus"
source /opt/Anaconda3-5.1.0-Linux-x86_64/bin/activate caiman || source activate caiman
ipcluster stop
sleep 5
ipcluster start --daemonize -n $1

bash: line 1: /opt/Anaconda3-5.1.0-Linux-x86_64/bin/activate: No such file or directory
2019-04-04 10:15:31.247 [IPClusterStop] CRITICAL | Could not read pid file, cluster is probably not running.


In [8]:
time.sleep(10)
# connect client
client = ipyparallel.Client()
time.sleep(2)
while len(client) < ncpus:
    sys.stdout.write(".")  # Give some visual feedback of things starting
    sys.stdout.flush()     # (de-buffered)
    time.sleep(0.5)

# create dview object
client.direct_view().execute('__a=1', block=True)
dview = client[:]
n_processes = len(client)
print('\n\nThe cluster appears to be setup. Number of parallel processes: %d' % (n_processes))



The cluster appears to be setup. Number of parallel processes: 4


### Map network drive
If the data is located on a network drive (i.e. Neurophysiology storage), we first need to connect the drive with the relevant user credentials.

In [9]:
connect_storage = config['data']['connect_storage']
if connect_storage:
    storage_user = config['data']['storage_user']
    storage_adress = config['data']['storage_adress']
    mountpoint = config['data']['mountpoint']
    storage_pw = getpass.getpass(prompt="Enter password for the remote storage")

In [10]:
# check if the mountpoint exists, if not create it
if connect_storage:
    if not os.path.isdir(mountpoint):
        os.makedirs(mountpoint)
    # list contents of the directory
    os.listdir(mountpoint)

In [11]:
%%bash -s "$connect_storage" "$storage_user" "$storage_pw" "$storage_adress" "$mountpoint"
if [ "$1" = 1 ]; then
    sudo mount -t cifs -o username=$2,password=$3,uid=$(id -u),gid=$(id -g) $4 $5
else
    echo "Not mounting storage"
fi

Not mounting storage


In [12]:
# list contents of the directory
if connect_storage:
    print('Mountpoint folder content:')
    print(os.listdir(mountpoint))

### Read other parameters from config file

In [13]:
# data parameters
data_folder = str(config['data']['data_folder'])
animal_folder = str(config['data']['animal_folder'])
day_folder = str(config['data']['day_folder'])
area_folder = str(config['data']['area_folder'])
data_folder = os.path.join(data_folder, animal_folder, day_folder, area_folder)
copy_to_temp = bool(config['data']['copy_to_temp'])

# analysis parameters
max_trials = config['analysis']['max_trials']
x_crop = config['analysis']['x_crop']
n_planes = config['analysis']['n_planes']
max_group_size = config['analysis']['max_group_size']

Copy the config file to the data folder, so that analysis parameters are preserved.

In [14]:
shutil.copy2(config_file, os.path.join(data_folder, config_file.replace('.yml', '_mc.yml')))

'/Users/Henry/Data/temp/Dendrites_Gwen/M5.2/20181211/S1/config_mc.yml'

### Copy data to temporary folder
If the data is stored on a remote network location, it is more robust to copy it to a local temporary folder for analysis. Afterwards, the results are copied back to the original location.

In [15]:
if copy_to_temp:
    # create a temp directory for analysis
    temp_dir = tempfile.mkdtemp()
    # create the data folder structure in the temporary directory
    temp_data_folder = os.path.join(temp_dir, animal_folder, day_folder, area_folder)
    os.makedirs(temp_data_folder, exist_ok=True)
    print('Created temporary analysis folder %s' % (temp_data_folder))

In [16]:
# select sessions for processing
p = re.compile('\d\d-\d\d-\d\d_Live') # regular expression that should match the folder names (ie. 01-23-45_Live)
sessions = [os.path.join(data_folder, x) for (i,x) in enumerate(sorted(os.listdir(data_folder))) if p.match(x) and i <= max_trials]

In [17]:
# copy data to temporary folder
if copy_to_temp:
    t_start = time.time()
    temp_sessions = dview.map_sync(utils.copyDirectory, sessions, [temp_data_folder]*len(sessions))
    # print elapsed time
    t_elapsed = time.time() - t_start
    print('Copied data to %s in %1.2f s (%1.2f s per session)' % (temp_dir, t_elapsed, t_elapsed/len(sessions)))

In [18]:
if copy_to_temp:
    sessions = temp_sessions
    working_data_folder = temp_data_folder
else:
    working_data_folder = data_folder

### Select TIFF files and extract metadata

In [19]:
tiff_files = []
xml_files = []
for i_session in sessions:
    tiff_files.append([os.path.join(i_session, x) for x in os.listdir(i_session) if x.endswith('.tif') and not 'stacked' in x][0])
    xml_files.append([os.path.join(i_session, x) for x in os.listdir(i_session) if 'parameters.xml' in x][0])

In [20]:
# read frame rate from parameters.xml
frame_rates = []
for ix, i_session in enumerate(sessions):
    tree = ET.parse(xml_files[ix])
    root = tree.getroot()
    for child in root:
        if child.tag == 'area0': # Note: only area 0!
            fr = child.find('Framerate_Hz')
            frame_rate = float(fr.text)
            frame_rates.append(frame_rate)
    if ix <= 10:
        print('Frame rate: %1.4f Hz (%s)' % (frame_rate, os.path.split(i_session)[1]))
if ix > 10:
    print('...')

Frame rate: 40.8319 Hz (09-52-17_Live)
Frame rate: 40.8319 Hz (09-52-31_Live)
Frame rate: 40.8319 Hz (09-52-45_Live)
Frame rate: 40.8319 Hz (09-52-59_Live)
Frame rate: 40.8319 Hz (09-53-11_Live)
Frame rate: 40.8319 Hz (09-53-25_Live)
Frame rate: 40.8319 Hz (09-53-39_Live)
Frame rate: 40.8319 Hz (09-53-51_Live)
Frame rate: 40.8319 Hz (09-54-03_Live)
Frame rate: 40.8319 Hz (09-54-17_Live)


### Convert TIFF files to ImageJ hyperstack files
This cell calls the `mosaicToStack` function in `utils` through the ipyparallel `map_sync` method to make use of multiple cores.

In [21]:
stacked_files = dview.map_sync(utils.mosaicToStack, tiff_files, [n_planes]*len(tiff_files), [x_crop]*len(tiff_files))

In [22]:
print('Processing %1.0f files:' % (len(stacked_files)))
print(*stacked_files[:10], sep='\n')
if len(stacked_files) > 10:
    print('...')
    print(*stacked_files[-5:], sep='\n')

Processing 10 files:
/Users/Henry/Data/temp/Dendrites_Gwen/M5.2/20181211/S1/09-52-17_Live/test_A0_Ch0_ 0033_stacked.tif
/Users/Henry/Data/temp/Dendrites_Gwen/M5.2/20181211/S1/09-52-31_Live/test_A0_Ch0_ 0034_stacked.tif
/Users/Henry/Data/temp/Dendrites_Gwen/M5.2/20181211/S1/09-52-45_Live/test_A0_Ch0_ 0035_stacked.tif
/Users/Henry/Data/temp/Dendrites_Gwen/M5.2/20181211/S1/09-52-59_Live/test_A0_Ch0_ 0036_stacked.tif
/Users/Henry/Data/temp/Dendrites_Gwen/M5.2/20181211/S1/09-53-11_Live/test_A0_Ch0_ 0037_stacked.tif
/Users/Henry/Data/temp/Dendrites_Gwen/M5.2/20181211/S1/09-53-25_Live/test_A0_Ch0_ 0038_stacked.tif
/Users/Henry/Data/temp/Dendrites_Gwen/M5.2/20181211/S1/09-53-39_Live/test_A0_Ch0_ 0039_stacked.tif
/Users/Henry/Data/temp/Dendrites_Gwen/M5.2/20181211/S1/09-53-51_Live/test_A0_Ch0_ 0040_stacked.tif
/Users/Henry/Data/temp/Dendrites_Gwen/M5.2/20181211/S1/09-54-03_Live/test_A0_Ch0_ 0041_stacked.tif
/Users/Henry/Data/temp/Dendrites_Gwen/M5.2/20181211/S1/09-54-17_Live/test_A0_Ch0_ 0042_s

### Join cropped TIF files
Next, we create a large joined TIF file from individual cropped files. Further processing will be done on the joined file.

In [23]:
n_groups = math.ceil(len(stacked_files) / float(max_group_size))
files_per_group = math.ceil(len(stacked_files) / n_groups)
stacked_files_by_group = []
print('Processing files in %d groups' % (n_groups))
for i_groups in range(int(n_groups)):
    start_ix = int(i_groups * files_per_group)
    stop_ix = int((i_groups+1) * files_per_group)
    stacked_files_by_group.append(stacked_files[start_ix:stop_ix])
    
    print('Group %d (%d - %d): %d files' % (i_groups+1, start_ix, stop_ix, len(stacked_files[start_ix:stop_ix])))

Processing files in 1 groups
Group 1 (0 - 10): 10 files


In [24]:
joined_tif_list = []
json_fname_list = []
total_frames_list = []
trial_indices_list = []
for i_group, stacked_files_group in enumerate(stacked_files_by_group):
    # load movies
    movies = cm.load_movie_chain(stacked_files_group, is3D=True, outtype=np.int16)
    
    total_frames = movies.shape[0]
    total_frames_list.append(total_frames)
    n_planes = movies.shape[1]
    dims = (movies.shape[2], movies.shape[3])
    
    frames_per_movie = dview.map_sync(utils.getFramesTif, stacked_files_group)
    
     # trial index for each frame
    trial_indices = []
    for i_frame, frame_count in enumerate(frames_per_movie):
        trial_indices = trial_indices + [i_frame]*frame_count
    trial_indices_list.append(trial_indices)
    
    for i_plane in range(n_planes):
        # derive joined file name and save
        joined_tif = '%s_%s_Join_G%d_F%d_P%d.tif' % (day_folder, area_folder, i_group, total_frames, i_plane)
        imsave(os.path.join(working_data_folder, joined_tif), movies[:, i_plane, :, :])
        print('Saved joined TIF file %s' % (joined_tif))
    
    movies = None # free the memory
    
    # create a Json file with information about source files
    meta = {"joined_file": joined_tif.replace('_P%d.tif' % (i_plane), ''), 
            "source_frames": frames_per_movie, 
            "source_file": [x.replace(working_data_folder + os.path.sep,'') for x in stacked_files_group],
            "trial_index": trial_indices,
            "frame_rate": frame_rate,
            "z_planes": n_planes
           }
    json_fname = joined_tif.replace('_P%d.tif' % (i_plane), '.json')
    with open(os.path.join(working_data_folder, json_fname), 'w') as fid:
        json.dump(meta, fid)
    
    # save output file names in list
    joined_tif_list.append(joined_tif.replace('_P%d.tif' % (i_plane), ''))
    json_fname_list.append(json_fname)
    
    print('Created JSON metadata file %s' % (json_fname))
    
# delete stacked TIF files (to save disk space)
dummy = dview.map_sync(os.remove, stacked_files)

100%|██████████| 10/10 [00:00<00:00, 18.03it/s]


Saved joined TIF file 20181211_S1_Join_G0_F707_P0.tif
Saved joined TIF file 20181211_S1_Join_G0_F707_P1.tif
Saved joined TIF file 20181211_S1_Join_G0_F707_P2.tif
Saved joined TIF file 20181211_S1_Join_G0_F707_P3.tif
Created JSON metadata file 20181211_S1_Join_G0_F707.json


### Display average signal intensity
This step is optional and useful as sanity check how the imported data looks like.

In [25]:
# # select group (0, 1, ...)
# group_ix = 0

# color_map = palettes.d3['Category10'][10] # colors for different planes

# # prepare data structure
# trial_names = [x.replace(working_data_folder + os.path.sep,'')[:8] for x in stacked_files_by_group[group_ix]]
# trial_names_frames = [trial_names[x] for x in trial_indices_list[group_ix]]
# data = {'x': np.array(range(total_frames_list[group_ix])), 
#         'trial_idx': trial_indices_list[group_ix],
#         'trial_name': trial_names_frames
#        }

# # add average for each plane
# for i_plane in range(n_planes):
#     tiff_file = os.path.join(working_data_folder, joined_tif_list[group_ix] + '_P%d.tif' % (i_plane))
#     mov = cm.load(tiff_file, outtype=np.int16)

#     # plot average signal intensity per frame
#     frame_avg = np.mean(np.mean(mov, axis=1), axis=1)

#     fieldname = 'y%s' % (i_plane)
#     data[fieldname] = frame_avg
    
# data_source = ColumnDataSource(data)

# # create figure and plot
# p = Figure(plot_width=900, plot_height=300, title=('Frame average - Group %d' % (group_ix))) 
# p.add_tools(CrosshairTool(), utils.getHover())
# for i_plane in range(n_planes):
#     p.line('x', 'y%s' % (i_plane), source=data_source, line_width=2, color=color_map[i_plane], legend='Plane %s' % (i_plane))

# show(p)

### Motion correction

First, setup the parameters for motion correction. The following parameters influence the **quality** of the motion correction:
- niter_rig ... number of iterations for rigid registration (larger = better). Little improvement likely above 5-10.
- strides ... intervals at which patches are laid out for motion correction (smaller = better)
- overlaps ... overlap between patches

Note that smaller values for strides / overlap will improve registration but also lead to NaNs in the output image. In general, there is a trade-off between the quality of registration and the presence / number of NaNs in the output (at least if there is significant motion).

In [26]:
# some parameters are derived directly from the data
max_shifts = (int(np.round(dims[0]/10)), int(np.round(dims[1]/10)))  # maximum allow rigid shift

In [27]:
for key in config['mc']:
    if isinstance(config['mc'][key], (list,)):
        # replace 'None' strings in lists with None
        config['mc'][key] = [None if x == 'None' else x for x in config['mc'][key]]
    elif isinstance(config['mc'][key], (str,)) and config['mc'][key] == 'None':
        config['mc'][key] = None

In [28]:
# parameters for motion correction
opts_dict = {
    # number of iterations rigid
    'niter_rig': config['mc']['niter_rig'],
    # run piecewise-rigid registration?
    'pw_rigid': config['mc']['pw_rigid'],
    'max_shifts': max_shifts,  # maximum allow rigid shift
    # for parallelization split the movies in num_splits chuncks across time
    'splits_rig': config['mc']['splits_rig'],
    # if none all the splits are processed and the movie is saved
    'num_splits_to_process_rig': config['mc']['num_splits_to_process_rig'],
    # intervals at which patches are laid out for motion correction
    'strides': config['mc']['strides'],
    # overlap between patches (size of patch strides+overlaps)
    'overlaps': config['mc']['overlaps'],
    # for parallelization split the movies in num_splits chuncks across time
    'splits_els': config['mc']['splits_els'],
    # if none all the splits are processed and the movie is saved
    'num_splits_to_process_els': config['mc']['num_splits_to_process_els'],
    'upsample_factor_grid': config['mc']['upsample_factor_grid'],  # upsample factor to avoid smearing when merging patches
    # maximum deviation allowed for patch with respect to rigid shift
    'max_deviation_rigid': config['mc']['max_deviation_rigid'],
    # Specifies how to deal with borders. (True, False, 'copy', 'min')
    'border_nan': config['mc']['border_nan'],
}
opts = params.CNMFParams(params_dict=opts_dict)

There are also some parameters for computing the quality metrics. These probably don't have to be changed.

In [29]:
# parameters for computing metrics
winsize = config['metrics']['winsize']
swap_dim = config['metrics']['swap_dim']
resize_fact_flow = config['metrics']['resize_fact_flow']    # downsample for computing ROF
iters_flow = config['metrics']['iters_flow'] # iterations for calculation of optic flow

Next, we define some functions. See the function doc strings for further information.

Now we are ready to run motion correction for the joined TIF file. If there are a lot of concatenated trials, this might take a while to complete.

The following outputs will be saved:
- result of rigid motion correction in Python mmap format and as TIF file
- result of pw-rigid motion correction in Python mmap format and as TIF file

### Correlation with template - uncorrected data
First, we just get the correlation with the template for the uncorrected data. This part is optional and can be used for example to re-assign groups.

In [30]:
# # Todo: parallelize
# corr_orig_list = []

# t_start = time.time()
# for ix_file, i_file in enumerate(joined_tif_list):
#     corr_orig_list.append([])
#     for i_plane in range(n_planes):
#         fname = os.path.join(working_data_folder, i_file + '_P%d.tif' % (i_plane))

#         # compute initial template by binned median filtering
#         template = cm.load(fname).bin_median(window=10)
    
#         corr_orig = []
#         for frame in cm.load(fname):
#             corr_orig.append(scipy.stats.pearsonr(frame.flatten(), template.flatten())[0])
#         corr_orig_list[ix_file].append(corr_orig)
    
# # print elapsed time
# t_elapsed = time.time() - t_start
# print('\nFinished pre-MC in %1.2f s (%1.2f s per frame)' % (t_elapsed, t_elapsed/(sum(total_frames_list)*n_planes)))

In [31]:
# # select group (0, 1, ...)
# group_ix = 0

# trial_names = [x.replace(working_data_folder + os.path.sep,'')[:8] for x in stacked_files_by_group[group_ix]]
# trial_names_frames = [trial_names[x] for x in trial_indices_list[group_ix]]

# # frames = np.array(range(len(corr_orig_list[group_ix])))
# data = {'x': np.array(range(len(corr_orig_list[group_ix][0]))), 
#         'trial_idx': trial_indices_list[group_ix],
#         'trial_name': trial_names_frames
#        }

# # add average for each plane
# for i_plane in range(n_planes):
#     fieldname = 'y%s' % (i_plane)
#     data[fieldname] = corr_orig_list[group_ix][i_plane]

# data_source = ColumnDataSource(data)

# p = Figure(plot_width=900, plot_height=300, title=('Correlation with template - Group %d' % (group_ix))) 
# p.add_tools(CrosshairTool(), utils.getHover())
# for i_plane in range(n_planes):
#     p.line('x', 'y%s' % (i_plane), source=data_source, line_width=2, color=color_map[i_plane], legend='Plane %s' % (i_plane))

# show(p)

### Run the motion correction
Next, we run the MC. This can be time consuming for large datasets.

In [32]:
t_start = time.time()
mc_list = []
for ix_file, i_file in enumerate(joined_tif_list):
    mc_list.append([])
    for i_plane in range(n_planes):
        fname = os.path.join(working_data_folder, i_file + '_P%d.tif' % (i_plane))
        
        if opts.get('motion', 'pw_rigid'):
            opts.set('motion', {'pw_rigid': False})
            mc = MotionCorrect(fname, dview=dview, **opts.get_group('motion'))
            mc.motion_correct(save_movie=True)
            fname_rig = mc.fname_tot_rig
            opts.set('motion', {'pw_rigid': True})
        
        mc = MotionCorrect(fname, dview=dview, **opts.get_group('motion'))
        mc.motion_correct(save_movie=True)
        
        if opts.get('motion', 'pw_rigid'):
            mc.fname_tot_rig = fname_rig
        
        mc_list[ix_file].append(mc)
        print('Finished MC for %s - Plane %d of %d' % (i_file, i_plane+1, n_planes))

# print elapsed time
t_elapsed = time.time() - t_start
print('\nFinished MC in %1.2f s (%1.2f s per frame)' % (t_elapsed, t_elapsed/(sum(total_frames_list)*n_planes)))

Finished MC for 20181211_S1_Join_G0_F707 - Plane 1 of 4
Finished MC for 20181211_S1_Join_G0_F707 - Plane 2 of 4
Finished MC for 20181211_S1_Join_G0_F707 - Plane 3 of 4
Finished MC for 20181211_S1_Join_G0_F707 - Plane 4 of 4

Finished MC in 93.48 s (0.03 s per frame)


### Assess quality of motion correction
A number of key metrics can be calculated to assess how much motion correction improved overall motion. 
1. Correlation
Correlations of each frame with the template image (binned median) for original, rigid correction and pw-rigid correction. The mean correlation gives an overall impression of motion. The minimum correlation indicates the parts of the movie that are worst affected by motion. Larger correlations indicate less motion.
2. Crispness
Crispness provides a measure of the smoothness of the corrected average image. Intuitively, a dataset with nonregistered motion will have a blurred mean image, resulting in a lower value for the total gradient field norm. Thus, larger values indicate a crisper average image and less residual motion. Crispness is calculated from the gradient field of the mean image (`np.gradient`).
3. Residual optical flow
Optic flow algorithms attempt to match each frame to the template by estimating locally smooth displacement fields. The output is an image where each pixel described the local displacement between template and frame at this point. The smaller the local displacement, the better the registration. Here we compute the matrix norm of the optic flow matrix as summary statistic.

In [33]:
# compute quality assessment metrics
crispness = []
norms = []
corr_mean = []
corr_min = []
metrics = []
for i_group in range(n_groups):
    crispness.append([])
    norms.append([])
    corr_mean.append([])
    corr_min.append([])
    metrics.append([])
    mc_group = mc_list[i_group]
    
    # remove Pool object (required for parallel processing)
    for ix, i_mc in enumerate(mc_group):
        i_mc.dview = None
        mc_group[ix] = i_mc
    
    metrics_list = dview.map_sync(mc_utils.computeMetrics, mc_group, [swap_dim]*len(mc_group), 
                                  [winsize]*len(mc_group), [resize_fact_flow]*len(mc_group), 
                                  [iters_flow]*len(mc_group))
    
    # collect results
    for mtrs in metrics_list:
        metrics[i_group].append(mtrs)
        if opts.get('motion', 'pw_rigid'):
            # correlations, crispness and norms of residual optic flow as indicators of registration quality
            crispness[i_group].append(np.array([mtrs['crispness_orig'], mtrs['crispness_rig'], mtrs['crispness_els']]))
            norms[i_group].append(np.array([np.mean(mtrs['norms_orig']), np.mean(mtrs['norms_rig']), np.mean(mtrs['norms_els'])]))
            corr_mean[i_group].append(np.array([np.mean(mtrs['corr_orig']), np.mean(mtrs['corr_rig']), np.mean(mtrs['corr_els'])]))
            corr_min[i_group].append(np.array([np.min(mtrs['corr_orig']), np.min(mtrs['corr_rig']), np.min(mtrs['corr_els'])]))
        else:
            crispness[i_group].append(np.array([mtrs['crispness_orig'], mtrs['crispness_rig']]))
            norms[i_group].append(np.array([np.mean(mtrs['norms_orig']), np.mean(mtrs['norms_rig'])]))
            corr_mean[i_group].append(np.array([np.mean(mtrs['corr_orig']), np.mean(mtrs['corr_rig'])]))
            corr_min[i_group].append(np.array([np.min(mtrs['corr_orig']), np.min(mtrs['corr_rig'])]))

clear_output()

Print different metrics for raw movie and rigid / pw-rigid corrected movies.

In [34]:
for i_group in range(n_groups):
    for i_plane in range(n_planes):
        print('MC evaluation - Group %d - Plane %d:' % (i_group, i_plane))
        mc_utils.printMetrics(corr_mean[i_group][i_plane], corr_min[i_group][i_plane], crispness[i_group][i_plane], norms[i_group][i_plane])
        print('\n')

MC evaluation - Group 0 - Plane 0:
Mean corr - raw / rigid: ['0.15', '0.22']
Min corr - raw / rigid: ['0.04', '0.09']
Crispness - raw / rigid: ['38', '59']
Norms - raw / rigid: ['6.06', '2.78']


MC evaluation - Group 0 - Plane 1:
Mean corr - raw / rigid: ['0.19', '0.27']
Min corr - raw / rigid: ['0.07', '0.14']
Crispness - raw / rigid: ['56', '90']
Norms - raw / rigid: ['12.25', '5.81']


MC evaluation - Group 0 - Plane 2:
Mean corr - raw / rigid: ['0.21', '0.29']
Min corr - raw / rigid: ['0.02', '0.04']
Crispness - raw / rigid: ['58', '96']
Norms - raw / rigid: ['16.23', '5.29']


MC evaluation - Group 0 - Plane 3:
Mean corr - raw / rigid: ['0.42', '0.59']
Min corr - raw / rigid: ['0.08', '0.20']
Crispness - raw / rigid: ['232', '377']
Norms - raw / rigid: ['169', '196']




#### Correlation with template image
Plot correlations of each frame with the template image (binned median) for original, rigid correction and pw-rigid correction. The bokeh plotting library provides a toolbar for interaction with the plot.

In [35]:
# # select group (0, 1, ...)
# group_ix = 0
# # select plane (0, 1, ..)
# plane_ix = 0

# p1 = Figure(plot_width=900, plot_height=300, title=('Correlation with template - Group %d - Plane %d' % (group_ix, plane_ix))) 
# frames = np.array(range(len(metrics[group_ix][plane_ix]['corr_orig'])))
# p1.line(frames,np.array(metrics[group_ix][plane_ix]['corr_orig']), line_width=2, legend='Original', color='blue')
# p1.line(frames,np.array(metrics[group_ix][plane_ix]['corr_rig']), line_width=2, legend='Rigid', color='orange')
# if opts.get('motion', 'pw_rigid'):
#     p1.line(frames,np.array(metrics[group_ix][plane_ix]['corr_els']), line_width=2, legend='PW-Rigid', color='green')

# p2 = Figure(plot_width=250, plot_height=250)
# p2.circle(np.array(metrics[group_ix][plane_ix]['corr_orig']), np.array(metrics[group_ix][plane_ix]['corr_rig']), size=5)
# p2.line([0,1],[0,1], line_width=1, color='black', line_dash='dashed')
# p2.xaxis.axis_label = 'Original'
# p2.yaxis.axis_label = 'Rigid'

# if opts.get('motion', 'pw_rigid'):
#     p3 = Figure(plot_width=250, plot_height=250)
#     p3.circle(np.array(metrics[group_ix][plane_ix]['corr_rig']), np.array(metrics[group_ix][plane_ix]['corr_els']), size=5)
#     p3.line([0,1],[0,1], line_width=1, color='black', line_dash='dashed')
#     p3.xaxis.axis_label = 'Rigid'
#     p3.yaxis.axis_label = 'PW-Rigid'

# # make a grid
# if opts.get('motion', 'pw_rigid'):
#     grid = gridplot([[p1, None], [p2, p3]], sizing_mode='fixed', toolbar_location='left')
# else:
#     grid = gridplot([[p1, p2], [None, None]], sizing_mode='fixed', toolbar_location='left')

# show(grid)

### Detect frames with bad motion
Identify frames with significant residual motion (low correlation with template). Write a JSON file with criterion and indices of frames matching the criterion. This file can be used in further analysis to exclude the frames corrupted by motion.

In [36]:
# thresh = [
#     [0, 0.1, 0.1, 0.1]
# ] # find frames where value is less than criterion (one value per group and plane)

# for i_group in range(n_groups):
#     for i_plane in range(n_planes):
#         print('Group %d - Plane %d' % (i_group, i_plane))
#         if opts.get('motion', 'pw_rigid'):
#             # pw-rigid registration
#             criterion = 'corr_els'
#             bad_frames = [ix for ix, i in enumerate(metrics[i_group][i_plane][criterion]) 
#                           if i < thresh[i_group][i_plane]]
#             print('%1.0f frames matching criterion after pw-rigid registration.' % (len(bad_frames)))
#             mc_utils.writeJsonBadFrames(criterion, thresh[i_group][i_plane], 
#                                         bad_frames, mc_list[i_group][i_plane], 'els', working_data_folder)
#         # rigid registration
#         criterion = 'corr_rig'
#         bad_frames = [ix for ix, i in enumerate(metrics[i_group][i_plane][criterion]) 
#                       if i < thresh[i_group][i_plane]]
#         print('\n%1.0f frames matching criterion after rigid registration.' % (len(bad_frames)))
#         mc_utils.writeJsonBadFrames(criterion, thresh[i_group][i_plane], 
#                                     bad_frames, mc_list[i_group][i_plane], 'rig', working_data_folder)
#         print('\n')

### Copy back results and delete the temporary folder
If copying fails for any reason (e.g. no more disk space), we can still recover the results from the temporary folder.

In [37]:
if copy_to_temp:
    file_types_to_copy = ['.tif', '.npz', '.json', '.mmap']
    files_to_copy = []
    for file_type in file_types_to_copy:
        file_list = [x for x in os.listdir(working_data_folder) if not os.path.isdir(x) and x.endswith(file_type)]
        files_to_copy = files_to_copy + file_list
    out = [shutil.copy2(os.path.join(working_data_folder, x), data_folder) for x in files_to_copy]
    
    # delete the temp. dir
    shutil.rmtree(temp_dir)

### Stop the cluster

In [38]:
%%bash
source /opt/Anaconda3-5.1.0-Linux-x86_64/bin/activate caiman || source activate caiman
ipcluster stop
sleep 1

bash: line 1: /opt/Anaconda3-5.1.0-Linux-x86_64/bin/activate: No such file or directory
2019-04-04 10:18:02.776 [IPClusterStop] Stopping cluster [pid=36194] with [signal=<Signals.SIGINT: 2>]
